In [93]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [94]:
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from tensorflow.keras.models import Sequential
import numpy as np

In [109]:
data = pd.read_csv('Final.csv')

print(data.head())



   0                                        review_body  sentiment
0  0                                                  0          0
1  1                 love watch keeps time wonderfully.          1
2  2                                          scratches          2
3  3  works well me. however, found cheaper prices p...          1
4  4  beautiful watch face. band looks nice around. ...          1


In [110]:
def preprocess_data(data):
    
    # Convert text to lowercase
    data['review_body'] = data['review_body'].str.strip().str.lower()
    return data

data = preprocess_data(data)
data=data.dropna()

In [111]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['review_body'] = data['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [112]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
data['review_body'] = data.review_body.apply(lemmatize_text)
print(data)

           0                                        review_body  sentiment
0          0                                                 0           0
1          1                 love watch keep time wonderfully.           1
2          2                                           scratch           2
3          3  work well me. however, found cheaper price pla...          1
4          4  beautiful watch face. band look nice around. l...          1
...      ...                                                ...        ...
15707  15707  purchased watch amazon promotion. would equall...          1
15708  15708  ordered watch wed. got sunday. watch rather ea...          1
15709  15709  love size perfect toddler four year old love it.           1
15710  15710                                              fine           1
15711  15711  experience material tool staff buy amazon neve...          1

[15691 rows x 3 columns]


In [113]:
reviews = data['review_body'].values
labels = data['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [114]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [131]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 200
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [136]:
max_words = 200
embd_len = 25
# Defining GRU model
gru_model = Sequential(name="GRU_Model")
gru_model.add(Embedding(vocab_size,
                        embd_len,
                        input_length=max_words))
gru_model.add(GRU(128,
                  activation='tanh',
                  return_sequences=False))
gru_model.add(Dense(1, activation='sigmoid'))
 
# Printing the Summary
print(gru_model.summary())
 
# Compiling the model
gru_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)
 
# Training the GRU model
num_epochs = 5
history = gru_model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)
 
# Printing model score on test data
print()


Model: "GRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_30 (Embedding)    (None, 200, 25)           75000     
                                                                 
 gru_30 (GRU)                (None, 128)               59520     
                                                                 
 dense_31 (Dense)            (None, 1)                 129       
                                                                 
Total params: 134,649
Trainable params: 134,649
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
331/331 [==============================] - 100s 294ms/step - loss: -0.1282 - accuracy: 0.7735 - val_loss: -0.3838 - val_accuracy: 0.7638
Epoch 2/5
331/331 [==============================] - 105s 316ms/step - loss: -0.5428 - accuracy: 0.7735 - val_loss: -0.4512 - val_accuracy: 0.7638
Epoch 3/5
331/33

NameError: name 'pred_labels' is not defined

In [138]:
prediction = gru_model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i == 1:
        pred_labels.append(1)
    elif i == 2:
        pred_labels.append(2)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

123/123 [==============================] - 9s 68ms/step
Accuracy of prediction on test set :  0.7723680856487382
